# Exercise: Visualization of Temperature Anomaly Map for a Given Date

In this exercise, you will use a previously downloaded ERA5 NetCDF dataset to:

- Compute the temperature anomaly over the whole spatial domain for a specific date.
- Plot the spatial anomaly on a map using **xarray** and **Cartopy**.

---

### Steps:

1. Load the NetCDF dataset with **xarray**.
2. Compute the daily climatology (mean temperature for each day of the year).
3. Calculate the anomaly by subtracting the climatology from the actual temperature data.
4. Select the spatial anomaly field for a chosen date.
5. Plot the spatial anomaly on a map using **Cartopy**, including coastlines and borders.

---

### Partial Code Template

```python
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pandas as pd

# Load dataset (replace with your downloaded filename)
ds = xr.open_dataset("your_downloaded_file.nc")

variable = "2m_temperature"

# Compute daily climatology (mean for each day of the year)
climatology = ds[variable].groupby("time.dayofyear").mean("time")

# Calculate anomaly
anomaly = ds[variable].groupby("time.dayofyear") - climatology

# Select the date to visualize
date_of_interest = pd.Timestamp("YYYY-MM-DD")  # Replace with your date
anomaly_on_date = anomaly.sel(time=date_of_interest)

# Plot the spatial anomaly
fig = plt.figure(figsize=(10, 6))
ax = plt.axes(projection=ccrs.PlateCarree())

anomaly_on_date.plot(
    ax=ax,
    cmap="coolwarm",
    cbar_kwargs={'label': 'Temperature Anomaly (°C)'},
    transform=ccrs.PlateCarree()
)

ax.coastlines()
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.set_title(f"Temperature Anomaly on {date_of_interest.date()}")
plt.show()
